# Distilbert with chunking
Note that not all the cells in this notebook has outputs shown, because it is converted to an `.py` file and run in the SoC slurm clusters, then the model and results are copied back. But the notebook can be run here.


This notebook requires this package from this branch to be installed to the python env
https://github.com/mim-solutions/bert_for_longer_texts/tree/add-multiclass-support

This notebook adapts code from the above repository (licensed under MIT license, license reproduced below)

The MIT License (MIT)

Copyright (c) 2023 MIM SOLUTIONS Sp. z o.o.

Initial project authors:
Michał Brzozowski, Marek Wachnicki

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
import string

import torch

torch.cuda.empty_cache()

import gc

gc.collect()

In [ ]:
import swifter

In [ ]:
SAVE_LOCATION = "results_distilbert_sliding_window_beltmulti"

from pathlib import Path

Path(SAVE_LOCATION).mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd
from io import StringIO

with open("raw_data/xtrain.txt", "r") as file:
    data = file.read()

text_stream = StringIO(data)

df = pd.read_csv(text_stream, sep='\t', header=None, names=['labels', 'text'])

In [ ]:
with open("raw_data/xdev.txt", "r") as file:
    data = file.read()

text_stream = StringIO(data)

val = pd.read_csv(text_stream, sep='\t', header=None, names=['labels', 'text'])

In [ ]:
balanced_test = pd.read_csv("raw_data/balancedtest.csv", names=['labels', 'text'])

In [ ]:
df['labels'] -= 1
val['labels'] -= 1
balanced_test['labels'] -= 1

In [ ]:
import re


def clean_text(text):
    pattern = r'(?<=[a-z])(?=[A-Z][a-z])'
    text = re.sub(pattern, ' ', text)

    return text


In [ ]:
def text_preprocess(text: str):
    cleaned_text = clean_text(text)
    return cleaned_text

In [ ]:
df_cleaned_text = df["text"].swifter.apply(text_preprocess)
df["text"] = df_cleaned_text

val_cleaned_text = val["text"].swifter.apply(text_preprocess)
val["text"] = val_cleaned_text

bt_cleaned_text = balanced_test["text"].swifter.apply(text_preprocess)
balanced_test["text"] = bt_cleaned_text

In [ ]:
from datasets import load_dataset
import numpy as np

In [ ]:
from belt_nlp.bert_classifier_with_pooling import BertClassifierWithPooling

MODEL_PARAMS = {
    "batch_size": 2,
    "learning_rate": 5e-5,
    "epochs": 3,
    "chunk_size": 510,
    "stride": 450,
    "minimal_chunk_length": 64,
    "pooling_strategy": "mean",
    "device": "cuda",
    "num_labels": 4,
    "pretrained_model_name_or_path": "distilbert/distilbert-base-cased"

}
model = BertClassifierWithPooling(**MODEL_PARAMS)

In [ ]:
model.fit(df["text"], df["labels"], epochs=3)  #  Warning about tokeninizing too long text is expected

In [ ]:
preds = model.predict(balanced_test["text"])
probabilities = model.predict_scores(balanced_test["text"])

In [ ]:
balanced_test["pred"] = preds

In [ ]:
balanced_test.to_csv(f"{SAVE_LOCATION}/predicted_results.csv", index=False)

In [ ]:
from sklearn.metrics import classification_report

cr = classification_report(balanced_test["labels"].astype(str), balanced_test["pred"].astype(str))

print(cr)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(balanced_test["labels"].astype(str), balanced_test["pred"].astype(str))
print(cm)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

cmd = ConfusionMatrixDisplay(cm)
cmd.plot()
